In [1]:
from pyspark.sql import SparkSession
def get_spark():
    spark = SparkSession.builder.appName('Spark_YT_Videos') \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.jars.packages",
                "org.apache.hadoop:hadoop-aws:3.2.2,"
                "com.amazonaws:aws-java-sdk-bundle:1.12.180") \
        .getOrCreate()
# This is mandate config on spark session to use AWS S3
    spark._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
    spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","com.amazonaws.auth.InstanceProfileCredentialsProvider,com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.s3a.impl", "org.apache.hadoop.fs.s3a.S3A")
# spark.sparkContext.setLogLevel("DEBUG")
    return spark
spark = get_spark()
print("Spark session created")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1666882784054_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark session created

In [8]:
from pyspark.sql import functions as f

# File location and type
file_location = "s3a://youtube-dataset-kshitij/input/videos/*.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
videos_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("schema",videos_schema) \
  .load(file_location)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
videos_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+----------------------+--------------------+-----------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|                 title|       channel_title|category_id|        publish_time|                          tags|   views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+----------------------+--------------------+-----------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|n1WpP7iowLc|     17.14.11|  Eminem - Walk On ...|          EminemVEVO|         10|2017-11-10T17:00:...|          "Eminem"|"Walk"|"...|17158579| 787425|   43420|       1258

In [10]:
videos_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)

In [11]:
videos_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

178363

In [12]:
all_videos_df = videos_df.drop("tags","thumbnail_link","comments_disabled","ratings_disabled","description","video_error_or_removed","video_id")
all_videos_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------------------+--------------------+-----------+--------------------+--------+-------+--------+-------------+
|trending_date|                 title|       channel_title|category_id|        publish_time|   views|  likes|dislikes|comment_count|
+-------------+----------------------+--------------------+-----------+--------------------+--------+-------+--------+-------------+
|     17.14.11|  Eminem - Walk On ...|          EminemVEVO|         10|2017-11-10T17:00:...|17158579| 787425|   43420|       125882|
|     17.14.11|  PLUSH - Bad Unbox...|           iDubbbzTV|         23|2017-11-13T17:00:...| 1014651| 127794|    1688|        13030|
|     17.14.11|  Racist Superman |...|        Rudy Mancuso|         23|2017-11-12T19:05:...| 3191434| 146035|    5339|         8181|
|     17.14.11|  I Dare You: GOING...|            nigahiga|         24|2017-11-12T18:01:...| 2095828| 132239|    1989|        17518|
|     17.14.11|  Ed Sheeran - Perf...|          Ed Sheeran|         1

In [13]:
all_videos_df = all_videos_df.withColumn("trending_date",f.to_date("trending_date",'yy.dd.MM')) \
   .withColumn("publish_time_1",f.regexp_replace("publish_time","T"," ")) \
   .withColumn("publish_time_2",f.regexp_replace("publish_time_1","Z","")) \
   .withColumn("publish_time_3",f.to_timestamp("publish_time_2",'yyyy-MM-dd HH:mm:ss.SSS'))

transformed_videos_df = all_videos_df.drop("publish_time","publish_time_1","publish_time_2").withColumnRenamed("publish_time_3","publish_time") \
.withColumn("publish_date",f.to_date("publish_time")) \
.select(all_videos_df.category_id.cast("int"),"title","channel_title","publish_date","publish_time","trending_date",all_videos_df.views.cast("int"),all_videos_df.likes.cast("int"),all_videos_df.dislikes.cast("int"),all_videos_df.comment_count.cast("int")) 

transformed_videos_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------------+--------------------+------------+-------------------+-------------+--------+-------+--------+-------------+
|category_id|                 title|       channel_title|publish_date|       publish_time|trending_date|   views|  likes|dislikes|comment_count|
+-----------+----------------------+--------------------+------------+-------------------+-------------+--------+-------+--------+-------------+
|         10|  Eminem - Walk On ...|          EminemVEVO|  2017-11-10|2017-11-10 17:00:03|   2017-11-14|17158579| 787425|   43420|       125882|
|         23|  PLUSH - Bad Unbox...|           iDubbbzTV|  2017-11-13|2017-11-13 17:00:00|   2017-11-14| 1014651| 127794|    1688|        13030|
|         23|  Racist Superman |...|        Rudy Mancuso|  2017-11-12|2017-11-12 19:05:24|   2017-11-14| 3191434| 146035|    5339|         8181|
|         24|  I Dare You: GOING...|            nigahiga|  2017-11-12|2017-11-12 18:01:41|   2017-11-14| 2095828| 132239|    1989|

In [14]:
transformed_videos_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- category_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- publish_date: date (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)

In [15]:
transformed_videos_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

178363

In [16]:
transformed_videos_df.write.parquet("s3a://youtube-dataset-kshitij/output/videos")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…